In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import re
import itertools
from tqdm import tqdm

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.manifold import TSNE
import networkx as nx

import tensorflow as tf
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import keras 
import scipy
from keras import backend as K
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Flatten,Activation,Dropout,Input,Embedding,RepeatVector,Permute,merge,Lambda
from keras.layers import multiply ,concatenate ,subtract,add,Add,Reshape                   
from keras.optimizers import RMSprop ,Adam
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Model

In [ ]:
train_data = pd.read_csv('/Users/s0c02nj/Desktop/Hike/train/train.csv')
train_feat = pd.read_csv('/Users/s0c02nj/Desktop/Hike/train/user_features.csv')

In [ ]:
test_data = pd.read_csv('/Users/s0c02nj/Desktop/Hike/test.csv')

In [ ]:
train_data.shape

Rows to remove

In [ ]:
rem_row = np.load('/Users/s0c02nj/Desktop/Hike/same_node_train_ids.npy')

In [ ]:
len(rem_row)

In [ ]:
train_data = train_data.drop(list(rem_row),axis=0)

In [ ]:
train_data.shape

In [ ]:
train_data_new = train_data[0:6975831]

All nodes are present

In [ ]:
train_data1 = train_data_new[['node1_id']]
train_data2 = train_data_new[['node2_id']]

y_train = train_data_new['is_chat']

In [ ]:
y_cat = to_categorical(y_train)

In [ ]:
test_data1 = test_data[['node1_id']]
test_data2 = test_data[['node2_id']]


Bringing the Train Features

In [ ]:
train_feat1 = pd.merge(left = train_data1, right= train_feat, how='left', 
                 left_on=['node1_id'], right_on=['node_id'])

In [ ]:
train_feat2 = pd.merge(left = train_data2, right= train_feat, how='left', 
                 left_on=['node2_id'], right_on=['node_id'])

Bringing Test Features

In [ ]:
test_feat1 = pd.merge(left = test_data1, right= train_feat, how='left', 
                 left_on=['node1_id'], right_on=['node_id'])

In [ ]:
test_feat2 = pd.merge(left = test_data2, right= train_feat, how='left', 
                 left_on=['node2_id'], right_on=['node_id'])

Standardizing Columns

In [ ]:
cols_std = ['f1','f2','f3','f4','f5','f6','f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13']

Importing the Node vectors

In [ ]:
path = '/Users/s0c02nj/Desktop/Hike/node_vec/node2vec.txt'

In [ ]:
count = 0
vectors_list = []

for o in tqdm(open(path)):
    
    values = o.split()
    values[0] = int(values[0])
    values[1:] = list(map(float, values[1:]))
    
    vectors_list.append(values)

In [ ]:
cols = []
for i in range(0,16):
    cols.append('data'+str(i))

In [ ]:
cols = ['node_id']+cols

In [ ]:
node_vec = pd.DataFrame(vectors_list)
node_vec.columns = cols

In [ ]:
node_vec = node_vec.dropna()

In [ ]:
node_vec.index = range(0,len(node_vec))

Joining with the node_vec features

In [ ]:
train_feat1_fin = pd.merge(left = train_feat1, right= node_vec, how='left', 
                 left_on=['node1_id'], right_on=['node_id'])

In [ ]:
train_feat2_fin = pd.merge(left = train_feat2, right= node_vec, how='left', 
                 left_on=['node2_id'], right_on=['node_id'])

In [ ]:
test_feat1_fin = pd.merge(left = test_feat1, right= node_vec, how='left', 
                 left_on=['node1_id'], right_on=['node_id'])

In [ ]:
test_feat2_fin = pd.merge(left = test_feat2, right= node_vec, how='left', 
                 left_on=['node2_id'], right_on=['node_id'])

In [ ]:
cols = []
for i in range(0,16):
    cols.append('data'+str(i))
cols_total = cols_std + cols

In [ ]:
train_feat1_fin = train_feat1_fin[cols_total]
train_feat2_fin = train_feat2_fin[cols_total]

In [ ]:
test_feat1_fin = test_feat1_fin[cols_total]
test_feat2_fin = test_feat2_fin[cols_total]

Deep Learning Archi

In [ ]:
def model_deep():
    
    #Defining the input-----> Continuous Variable
    inputs1 = Input(shape=(29,))
    
    #Defining the input-----> Categorical Variable
    inputs2 = Input(shape=(29,))
    
    #Defining the -----> Complaint-reason
    layer_merge = concatenate([inputs1,inputs2])
    
    #Dense Layer-----> This will be the Weight Matrix W (n*k)
    layer_dense = Dense(100, activation='relu')(layer_merge)
    layer_dense = Dropout(0.5)(layer_dense)
    layer_dense = Dense(60, activation='relu')(layer_merge)
    layer_dense = Dropout(0.5)(layer_dense)
    layer_dense = Dense(40, activation='relu')(layer_merge)
    layer_dense = Dropout(0.5)(layer_dense)
    layer_dense = Dense(20, activation='relu')(layer_merge)
    layer_dense = Dropout(0.5)(layer_dense)
    layer_dense = Dense(10, activation='relu')(layer_merge)
    
    
    #Total Loss to Minimize
    loss_total = Dense(2,activation='softmax')(layer_dense)
    
    
    model = Model(inputs=[inputs1,inputs2], outputs= loss_total)
    return model

In [ ]:
model = model_deep()
model.summary()

In [ ]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [ ]:
model.compile(loss = "binary_crossentropy", optimizer = Adam(lr=0.001), metrics = [auc])

In [ ]:
#Deep Learning Model
history = model.fit([train_feat1_fin,
                     train_feat2_fin] ,
                     y_cat, batch_size = 1024, epochs = 10, validation_split=0.1,
                     class_weight = {0:2 ,1:40}, 
                     verbose = 1)


In [ ]:
#Tets
y_prob = model.predict([test_feat1_fin,test_feat2_fin])


In [ ]:
y_ones = y_prob[:,1]

In [ ]:
sub_df = test_data.copy()

In [ ]:
sub_df['is_chat'] = y_ones

In [ ]:
sub_df = sub_df[['id','is_chat']]

In [ ]:
z = np.load('/Users/s0c02nj/Desktop/Hike/same_node_test_ids.npy')
z_set = set(z)

In [ ]:
out = []
for i in tqdm(range(0,len(sub_df))):
    
    i_id = sub_df['id'].iloc[i]
    
    if i_id in z_set:
        k=0
    else:
        k = sub_df['is_chat'].iloc[i]
        
    out.append(k)

In [ ]:
sub_df['is_chat'] = out

In [ ]:
sub_df.to_csv('/Users/s0c02nj/Desktop/Hike/Submission6.csv',index=False)